### Retrieve the Products complete data from e-katanalotis.gr
#### Date range: 26/07/2024  -  30/07/2024

#### Convert the JSON to a DataFrame through json library

In [2]:
import json

# Load the JSON data from the file
with open('categories_suppliers_merchats_products_29-07-2024.json', 'r', encoding='utf-8') as file: 
    data = json.load(file)

In [3]:
import pandas as pd

In [25]:
# Load the categories & suppliers from the CSV file
categories_file_path = "df_categories_26-30 July 2024.csv"
suppliers_file_path = 'df_suppliers_26-30 July 2024.csv'
df_suppliers = pd.read_csv(suppliers_file_path)
df_categories = pd.read_csv(categories_file_path)

In [26]:
df_suppliers.head()

,id,name
0,0,ΥΦΑΝΤΗΣ
1,1,ΕΝ ΕΛΛΑΔΙ
2,2,ΝΙΚΑΣ
3,3,CRETA FARMS
4,4,TOSTAKI


In [27]:
df_categories.head()

,sub_sub_category_name,sub_sub_category_uuid,sub_category_name,sub_category_uuid,parent_category_name,parent_category_uuid
0,Λαδερά,3,Λαδερά,2,Έτοιμα Γεύματα,1
1,Αλλαντικά Γαλοπούλας,6,Αλλαντικά Γαλοπούλας & Κοτόπουλου,5,Αλλαντικά,4
2,Αλλαντικά Κοτόπουλου,7,Αλλαντικά Γαλοπούλας & Κοτόπουλου,5,Αλλαντικά,4
3,Ζαμπόν Μπούτι,9,"Ζαμπόν, Μπέικον & Ωμοπλάτη",8,Αλλαντικά,4
4,Προσούτο & Jamon,10,"Ζαμπόν, Μπέικον & Ωμοπλάτη",8,Αλλαντικά,4


In [28]:
# Convert category UUID columns to string for consistent merging
df_categories['sub_sub_category_uuid'] = df_categories['sub_sub_category_uuid'].astype(str)
df_categories['sub_category_uuid'] = df_categories['sub_category_uuid'].astype(str)
df_categories['parent_category_uuid'] = df_categories['parent_category_uuid'].astype(str)

# Convert supplier UUID column to string for consistent merging
df_suppliers['id'] = df_suppliers['id'].astype(str)

In [29]:
# Initialize an empty list to store data
all_product_data = []

# Extract the products data
products_data = data['context']['MAPP_PRODUCTS']['result']['products']

# Iterate through the products and expand prices
for product in products_data:
    barcode = product.get('barcode')
    name = product.get('name')
    supplier_id = product.get('supplier')
    if supplier_id is not None:
        supplier_id = str(supplier_id)
    category_uuids = product.get('category', [])  # List of category numbers
    
     # Initialize category information
    category_info = {
        'parent_category_uuid': None,
        'parent_category_name': None,
        'sub_category_uuid': None,
        'sub_category_name': None,
        'sub_sub_category_uuid': None,
        'sub_sub_category_name': None
    }
    
    # Match each category number to the corresponding names using df_categories
    for category_uuid in category_uuids:
        matched_category = df_categories[
            (df_categories['sub_sub_category_uuid'] == str(category_uuid)) |
            (df_categories['sub_category_uuid'] == str(category_uuid)) |
            (df_categories['parent_category_uuid'] == str(category_uuid))
        ]
        if not matched_category.empty:
            # Assign category information based on the match
            if str(category_uuid) in matched_category['sub_sub_category_uuid'].values:
                category_info['sub_sub_category_uuid'] = category_uuid
                category_info['sub_sub_category_name'] = matched_category['sub_sub_category_name'].values[0]
            elif str(category_uuid) in matched_category['sub_category_uuid'].values:
                category_info['sub_category_uuid'] = category_uuid
                category_info['sub_category_name'] = matched_category['sub_category_name'].values[0]
            elif str(category_uuid) in matched_category['parent_category_uuid'].values:
                category_info['parent_category_uuid'] = category_uuid
                category_info['parent_category_name'] = matched_category['parent_category_name'].values[0]
    
    # Flatten the prices data
    for price_entry in product.get('prices', []):
        price_entry['barcode'] = barcode
        price_entry['name'] = name
        price_entry['supplier_uuid'] = supplier_id
        price_entry.update(category_info)  # Add category information to each price entry
        all_product_data.append(price_entry)

# Create a DataFrame from the list
df_products_expanded = pd.json_normalize(all_product_data)

# Select and reorder the relevant columns
df_products_expanded = df_products_expanded[[
    'barcode', 'name', 'supplier_uuid', 'merchant_uuid', 'price', 'price_normalized',
    'parent_category_uuid', 'parent_category_name',
    'sub_category_uuid', 'sub_category_name',
    'sub_sub_category_uuid', 'sub_sub_category_name'
]]

In [31]:
# Extract the merchants data
merchants_data = data['context']['MAPP_PRODUCTS']['result']['merchants']

# Create a DataFrame from the merchants data
df_merchants = pd.DataFrame(merchants_data)

# Merge the df_products_expanded and df_merchants based on the merchant_uuid
df_merged = pd.merge(df_products_expanded, df_merchants, on='merchant_uuid', how='left')
df_merged.head()

,barcode,name_x,supplier_uuid,merchant_uuid,price,price_normalized,parent_category_uuid,parent_category_name,sub_category_uuid,sub_category_name,sub_sub_category_uuid,sub_sub_category_name,name_y,image,display_name
0,1001,ΠΙΝΔΟΣ ΚΟΤΟΠΟΥΛΟ ΕΛΛΗΝΙΚΟ ΝΩΠΟ(ΤΙΜ. ΚΙΛ),486,1.0,3.75,3.75,410,Φρέσκο Κρέας,414,Φρέσκα Πουλερικά,417,Κοτόπουλο ολόκληρο,masoutis,masoutis.png,Μασούτης
1,1001,ΠΙΝΔΟΣ ΚΟΤΟΠΟΥΛΟ ΕΛΛΗΝΙΚΟ ΝΩΠΟ(ΤΙΜ. ΚΙΛ),486,4.0,3.09,3.09,410,Φρέσκο Κρέας,414,Φρέσκα Πουλερικά,417,Κοτόπουλο ολόκληρο,mymarket,mymarket.png,My Market
2,1001,ΠΙΝΔΟΣ ΚΟΤΟΠΟΥΛΟ ΕΛΛΗΝΙΚΟ ΝΩΠΟ(ΤΙΜ. ΚΙΛ),486,7.0,2.98,2.98,410,Φρέσκο Κρέας,414,Φρέσκα Πουλερικά,417,Κοτόπουλο ολόκληρο,synka,synka.png,Synka
3,1001,ΠΙΝΔΟΣ ΚΟΤΟΠΟΥΛΟ ΕΛΛΗΝΙΚΟ ΝΩΠΟ(ΤΙΜ. ΚΙΛ),486,8.0,2.98,2.98,410,Φρέσκο Κρέας,414,Φρέσκα Πουλερικά,417,Κοτόπουλο ολόκληρο,xalkiadakis,xalkiadakis.png,Xalkiadakis
4,1001,ΠΙΝΔΟΣ ΚΟΤΟΠΟΥΛΟ ΕΛΛΗΝΙΚΟ ΝΩΠΟ(ΤΙΜ. ΚΙΛ),486,9.0,3.00,3.00,410,Φρέσκο Κρέας,414,Φρέσκα Πουλερικά,417,Κοτόπουλο ολόκληρο,sklavenitis,sklavenitis.png,Σκλαβενίτης


In [36]:
df_merged = df_merged.rename(columns={'name_x':'name'})
df_merged = df_merged.rename(columns={'name_y':'merchant_name'})
df_merged = df_merged.rename(columns={'display_name':'merchant_displayname'})


# Select and reorder the relevant columns
df_merged = df_merged[[
    'barcode', 'name', 'supplier_uuid', 'price', 'price_normalized',
    'parent_category_uuid', 'parent_category_name',
    'sub_category_uuid', 'sub_category_name',
    'sub_sub_category_uuid', 'sub_sub_category_name','merchant_uuid', 'merchant_name',  'merchant_displayname','image'
]]

df_merged.head()

,barcode,name,supplier_uuid,price,price_normalized,parent_category_uuid,parent_category_name,sub_category_uuid,sub_category_name,sub_sub_category_uuid,sub_sub_category_name,merchant_uuid,merchant_name,merchant_displayname,image
0,1001,ΠΙΝΔΟΣ ΚΟΤΟΠΟΥΛΟ ΕΛΛΗΝΙΚΟ ΝΩΠΟ(ΤΙΜ. ΚΙΛ),486,3.75,3.75,410,Φρέσκο Κρέας,414,Φρέσκα Πουλερικά,417,Κοτόπουλο ολόκληρο,1.0,masoutis,Μασούτης,masoutis.png
1,1001,ΠΙΝΔΟΣ ΚΟΤΟΠΟΥΛΟ ΕΛΛΗΝΙΚΟ ΝΩΠΟ(ΤΙΜ. ΚΙΛ),486,3.09,3.09,410,Φρέσκο Κρέας,414,Φρέσκα Πουλερικά,417,Κοτόπουλο ολόκληρο,4.0,mymarket,My Market,mymarket.png
2,1001,ΠΙΝΔΟΣ ΚΟΤΟΠΟΥΛΟ ΕΛΛΗΝΙΚΟ ΝΩΠΟ(ΤΙΜ. ΚΙΛ),486,2.98,2.98,410,Φρέσκο Κρέας,414,Φρέσκα Πουλερικά,417,Κοτόπουλο ολόκληρο,7.0,synka,Synka,synka.png
3,1001,ΠΙΝΔΟΣ ΚΟΤΟΠΟΥΛΟ ΕΛΛΗΝΙΚΟ ΝΩΠΟ(ΤΙΜ. ΚΙΛ),486,2.98,2.98,410,Φρέσκο Κρέας,414,Φρέσκα Πουλερικά,417,Κοτόπουλο ολόκληρο,8.0,xalkiadakis,Xalkiadakis,xalkiadakis.png
4,1001,ΠΙΝΔΟΣ ΚΟΤΟΠΟΥΛΟ ΕΛΛΗΝΙΚΟ ΝΩΠΟ(ΤΙΜ. ΚΙΛ),486,3.00,3.00,410,Φρέσκο Κρέας,414,Φρέσκα Πουλερικά,417,Κοτόπουλο ολόκληρο,9.0,sklavenitis,Σκλαβενίτης,sklavenitis.png


In [37]:
# Merge the df_merged with df_suppliers to add supplier names
df_final = pd.merge(df_merged, df_suppliers, left_on='supplier_uuid', right_on='id', how='left')

# Drop the extra 'id' column from the suppliers DataFrame
df_final = df_final.drop(columns=['id'])

df_final.head()

,barcode,name_x,supplier_uuid,price,price_normalized,parent_category_uuid,parent_category_name,sub_category_uuid,sub_category_name,sub_sub_category_uuid,sub_sub_category_name,merchant_uuid,merchant_name,merchant_displayname,image,name_y
0,1001,ΠΙΝΔΟΣ ΚΟΤΟΠΟΥΛΟ ΕΛΛΗΝΙΚΟ ΝΩΠΟ(ΤΙΜ. ΚΙΛ),486,3.75,3.75,410,Φρέσκο Κρέας,414,Φρέσκα Πουλερικά,417,Κοτόπουλο ολόκληρο,1.0,masoutis,Μασούτης,masoutis.png,ΠΙΝΔΟΣ
1,1001,ΠΙΝΔΟΣ ΚΟΤΟΠΟΥΛΟ ΕΛΛΗΝΙΚΟ ΝΩΠΟ(ΤΙΜ. ΚΙΛ),486,3.09,3.09,410,Φρέσκο Κρέας,414,Φρέσκα Πουλερικά,417,Κοτόπουλο ολόκληρο,4.0,mymarket,My Market,mymarket.png,ΠΙΝΔΟΣ
2,1001,ΠΙΝΔΟΣ ΚΟΤΟΠΟΥΛΟ ΕΛΛΗΝΙΚΟ ΝΩΠΟ(ΤΙΜ. ΚΙΛ),486,2.98,2.98,410,Φρέσκο Κρέας,414,Φρέσκα Πουλερικά,417,Κοτόπουλο ολόκληρο,7.0,synka,Synka,synka.png,ΠΙΝΔΟΣ
3,1001,ΠΙΝΔΟΣ ΚΟΤΟΠΟΥΛΟ ΕΛΛΗΝΙΚΟ ΝΩΠΟ(ΤΙΜ. ΚΙΛ),486,2.98,2.98,410,Φρέσκο Κρέας,414,Φρέσκα Πουλερικά,417,Κοτόπουλο ολόκληρο,8.0,xalkiadakis,Xalkiadakis,xalkiadakis.png,ΠΙΝΔΟΣ
4,1001,ΠΙΝΔΟΣ ΚΟΤΟΠΟΥΛΟ ΕΛΛΗΝΙΚΟ ΝΩΠΟ(ΤΙΜ. ΚΙΛ),486,3.00,3.00,410,Φρέσκο Κρέας,414,Φρέσκα Πουλερικά,417,Κοτόπουλο ολόκληρο,9.0,sklavenitis,Σκλαβενίτης,sklavenitis.png,ΠΙΝΔΟΣ


In [42]:
df_final = df_final.rename(columns={'name_y':'supplier_name'})
df_final = df_final.rename(columns={'name_x':'name'})

In [43]:
# Select and reorder the relevant columns
df_final = df_final[[
    'barcode', 'name',  'price', 'price_normalized',
    'parent_category_uuid', 'parent_category_name',
    'sub_category_uuid', 'sub_category_name',
    'sub_sub_category_uuid', 'sub_sub_category_name','merchant_uuid', 'merchant_name',  'merchant_displayname','supplier_uuid', 'supplier_name','image'
]]

df_final.head()

,barcode,name,price,price_normalized,parent_category_uuid,parent_category_name,sub_category_uuid,sub_category_name,sub_sub_category_uuid,sub_sub_category_name,merchant_uuid,merchant_name,merchant_displayname,supplier_uuid,supplier_name,image
0,1001,ΠΙΝΔΟΣ ΚΟΤΟΠΟΥΛΟ ΕΛΛΗΝΙΚΟ ΝΩΠΟ(ΤΙΜ. ΚΙΛ),3.75,3.75,410,Φρέσκο Κρέας,414,Φρέσκα Πουλερικά,417,Κοτόπουλο ολόκληρο,1.0,masoutis,Μασούτης,486,ΠΙΝΔΟΣ,masoutis.png
1,1001,ΠΙΝΔΟΣ ΚΟΤΟΠΟΥΛΟ ΕΛΛΗΝΙΚΟ ΝΩΠΟ(ΤΙΜ. ΚΙΛ),3.09,3.09,410,Φρέσκο Κρέας,414,Φρέσκα Πουλερικά,417,Κοτόπουλο ολόκληρο,4.0,mymarket,My Market,486,ΠΙΝΔΟΣ,mymarket.png
2,1001,ΠΙΝΔΟΣ ΚΟΤΟΠΟΥΛΟ ΕΛΛΗΝΙΚΟ ΝΩΠΟ(ΤΙΜ. ΚΙΛ),2.98,2.98,410,Φρέσκο Κρέας,414,Φρέσκα Πουλερικά,417,Κοτόπουλο ολόκληρο,7.0,synka,Synka,486,ΠΙΝΔΟΣ,synka.png
3,1001,ΠΙΝΔΟΣ ΚΟΤΟΠΟΥΛΟ ΕΛΛΗΝΙΚΟ ΝΩΠΟ(ΤΙΜ. ΚΙΛ),2.98,2.98,410,Φρέσκο Κρέας,414,Φρέσκα Πουλερικά,417,Κοτόπουλο ολόκληρο,8.0,xalkiadakis,Xalkiadakis,486,ΠΙΝΔΟΣ,xalkiadakis.png
4,1001,ΠΙΝΔΟΣ ΚΟΤΟΠΟΥΛΟ ΕΛΛΗΝΙΚΟ ΝΩΠΟ(ΤΙΜ. ΚΙΛ),3.00,3.00,410,Φρέσκο Κρέας,414,Φρέσκα Πουλερικά,417,Κοτόπουλο ολόκληρο,9.0,sklavenitis,Σκλαβενίτης,486,ΠΙΝΔΟΣ,sklavenitis.png


In [44]:
# save the DataFrame to a CSV file
df_final.to_csv('df_products_alldata_26-30 July 2024.csv', index=False)